In [ ]:
import numpy as np
import torch

import time
from Utilities import Utilities as Utils
from IPython.display import clear_output

In [ ]:
Filters = 128
Layers = 13
HistoryDepth = 8
KernalSize = 3

ResNet, PolHead, ValHead = Utils.modelLoader('test1', Filters, Layers, HistoryDepth, 'cpu')

In [ ]:
board = np.zeros((HistoryDepth + 1, 15, 15), dtype=bool)

In [ ]:
player = 0
while (True):
    if player % 2:
        modelInput = torch.from_numpy(board.astype(np.float32)).unsqueeze(0)

        # Model
        with torch.no_grad():
            resnetOut = ResNet(modelInput)
            policyOut = PolHead(resnetOut)
            valueOut = ValHead(resnetOut)
  
        policyOut = torch.nn.functional.softmax(policyOut, dim=1)
        policyOut = np.array(policyOut)
        policyOut = policyOut.reshape(15, 15)
        policyOut *= ~board[HistoryDepth]
        policyOut *= ~board[HistoryDepth // 2]
        policyOut = policyOut.flatten()

        valueOut = valueOut[0].item()

        index = policyOut.argmax()
        x, y = int(index // 15), int(index % 15)
        board = Utils.makeMove(board, x, y)
        clear_output(wait=True)
        print(x,y)
        print(Utils.sliceGamestate(board, 0))
        print(valueOut)
        time.sleep(1)
    else:
        x = int(input("X:"))
        y = int(input("Y:"))

        board = Utils.makeMove(board, x, y)
        clear_output(wait=True)
        print(Utils.sliceGamestate(board, 0))
        time.sleep(1)
    player += 1